## Polytope Climate-DT Time Series example notebook

This notebook shows how to use earthkit-data and earthkit-plots to pull destination-earth data from LUMI and plot it using earthkit-plots.

Before running the notebook you need to set up your credentials. See the main readme of this repository for different ways to do this or use the following cells to authenticate.

You will need to generate your credentials using the desp-authentication.py script.

This can be run as follows:

In [ ]:
!python3 ~/polytope-examples/desp-authentication.py -u <user_name> -p <password>

This will generate a token that can then be used by earthkit and polytope.

You can also run the script without arguments to enter your username and password.

In [ ]:
!python3 ~/polytope-examples/desp-authentication.py 

# Requirements
To run this notebook install the following:
* pip install earthkit-data
* pip install earthkit-plots
* pip install earthkit-regrid  (Optional for spectral variables)
* pip install cf-units         (Optional for unit conversion in maps)

If you do not have eccodes installed please install eccodes using conda as it is a dependency of earthkit, or install earthkit via conda

* conda install eccodes -c conda-forge
* conda install earthkit-data -c conda-forge

In [ ]:
# This request matches multiple parameter of the climate DT
 
import earthkit.data

LOCATION = ((38, -9.5))


request = {
    "activity": "scenariomip",
    "class": "d1",
    "dataset": "climate-dt",
    "experiment": "ssp3-7.0",
    "generation": "1",
    "levtype": "sfc",
    "date": "20210101/to/20210110",
    "model": "ifs-nemo",
    "expver": "0001",
    "param": "167",
    "realization": "1",
    "resolution": "high",
    "stream": "clte",
    "type": "fc",
    "time": "0000",
    #"time": "0000/0100/0200/0300/0400/0500/0600/0700/0800/0900/1000/1100/1200/1300/1400/1500/1600/1700/1800/1900/2000/2100/2200/2300",
    "feature": {
        "type" : "timeseries",
        "points": [[LOCATION[0], LOCATION[1]]],
        "time_axis": "date",
    }
}

ds = earthkit.data.from_source("polytope", "destination-earth", request, stream=False, address='polytope-test.lumi.apps.dte.destination-earth.eu')


In [ ]:
ds._json()

In [18]:
def location_to_string(location):
    """
    Converts latitude and longitude to a string representation with degrees
    and N/S/E/W.
    """
    (lat, lon) = location
    lat_dir = "N" if lat >= 0 else "S"
    lon_dir = "E" if lon >= 0 else "W"
    return f"{abs(lat):.2f}°{lat_dir}, {abs(lon):.2f}°{lon_dir}"

In [15]:
da = ds.to_xarray()
da

<xarray.Dataset> Size: 272B
Dimensions:   (x: 1, y: 1, z: 1, number: 1, datetime: 1, t: 10)
Coordinates:
  * x         (x) float64 8B 37.97
  * y         (y) float64 8B 350.5
  * z         (z) int64 8B 0
  * number    (number) int64 8B 0
  * datetime  (datetime) <U20 80B '2021-01-10T00:00:00Z'
  * t         (t) datetime64[ns] 80B 2021-01-01 2021-01-02 ... 2021-01-10
Data variables:
    2t        (x, y, z, number, datetime, t) float64 80B 287.8 286.2 ... 288.3
Attributes: (12/14)
    activity:       scenariomip
    class:          d1
    dataset:        climate-dt
    Forecast date:  2021-01-10T00:00:00Z
    experiment:     ssp3-7.0
    expver:         0001
    ...             ...
    model:          ifs-nemo
    realization:    1
    resolution:     high
    stream:         clte
    type:           fc
    number:         0

In [19]:
from earthkit.plots.interactive import Chart

TIME_FREQUENCY = "1d"
QUANTILES = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]

chart = Chart()
chart.title(f"ECMWF ensemble meteogram at {location_to_string(LOCATION)}")
#chart.box(ds, time_frequency=TIME_FREQUENCY, quantiles=QUANTILES)
chart.line(ds, line_color='grey', time_frequency=TIME_FREQUENCY)
chart.show()  # Replace with chart.show() in an interactive session!